In [1]:
#mount drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cd into you director

%cd '/content/drive/MyDrive/Roozbeh'

/content/drive/MyDrive/Roozbeh


In [3]:
import numpy as np
import os
import PIL
import PIL.Image
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow as tf
import datetime
from numpy import random

In [4]:
#load pre-trained model (check tf for avaiable models)
# we will use InceptionV3 here
# base_model = InceptionV3(weights='imagenet', include_top=False)
base_model = tf.keras.applications.ResNet101V2(weights='imagenet', include_top=False)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.00005,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
  
)

loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

171317808/171317808 [==============================] - 1s 0us/step


In [5]:
#add classification layers on top of the pre-trained model

# add a global spatial average pooling layer
def create_model():
    
  x = base_model.output
  x = GlobalAveragePooling2D()(x) 
  # let's add a fully-connected layer
  x = Dense(64, activation='relu')(x)
  # x = layers.Dropout(.2)(x)
  x = Dense(32, activation='relu')(x)
  # x = layers.Dropout(.2)(x)
  predictions = Dense(6, activation='softmax')(x)
  
  for layer in base_model.layers:
    layer.trainable = False
  # this is the model we will train

  model = Model(inputs=base_model.input, outputs=predictions)

  # model.compile(optimizer=optimizer,
  #               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  #               metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

In [6]:
def compile_model(model,optimizer,loss):
  model.compile(optimizer=optimizer,
                loss=loss,
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                # callbacks=[cp_callback],
                )
  return model

In [7]:
new_model = tf.keras.models.load_model('bestmodel/resnet/')
#save_model_path = '/content/drive/MyDrive/Roozbeh/bestmodel/resnet/saved_model.pb'


In [9]:
import datetime
import matplotlib.pyplot as plt
#write prediction code 

import json
# with open('CRC1_data.json','r') as file:
with open('/content/drive/MyDrive/Roozbeh/CRC1_data.json','r') as file:  
  data=json.load(file)
#load image
print(len(data['image']))
print(len(data['Depth']))
def find_closest_value(vector,value):
    temp = [abs(i-value) for i in vector]
    indx = temp.index(min(temp))
    return vector[indx], indx
D= [float(i[1]) for i in np.array(data['Depth'])]  

680902
680902


**Load data from other facies. this is testing a single image**

In [ ]:
value, indx = find_closest_value(D,1184.5)   
print(value)
print(indx) 
temp_img = np.array(data['image'][indx:indx+192])
print(np.shape(temp_img))
fig, ax = plt.subplots()
im = ax.imshow(temp_img)
# fig.colorbar(im)
plt.show()

In [ ]:
def create_samples_from_idx(facies_data,index=0):
  sample_idx = index
  # this should be converted into a new function / so the function should be classed insude the for loop 
  image_data = facies_data[sample_idx:sample_idx+192] 

  image = np.array(image_data)
  image = image/128
  image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
  image4d = np.expand_dims(image, axis=0)
  # print('shape of the created image is {}'.format(np.shape(image)))
  return image,image4d
# normalization_layer = layers.Rescaling(1./255)

temp_img,temp_img4d = create_samples_from_idx(data['image'],indx)
print(np.shape(temp_img4d))
# normalized_img = temp_img.map(lambda x, y: (normalization_layer(x), y))

fig, ax = plt.subplots()
im = ax.imshow(temp_img)


In [ ]:
print(new_model.predict(temp_img4d))
print()
print(f"predicted class is {1+np.argmax(new_model.predict(temp_img4d))}")

**evaluate a full log from here on**

In [ ]:
def create_samples_from_idx(facies_data,index=0):
  sample_idx = index
  # this should be converted into a new function / so the function should be classed insude the for loop 
  image_data = facies_data[sample_idx:sample_idx+192] 

  image = np.array(image_data)
  image = image/128
  image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
  image4d = np.expand_dims(image, axis=0)
  # print('shape of the created image is {}'.format(np.shape(image)))
  return image,image4d
# normalization_layer = layers.Rescaling(1./255)


def init_list_of_objects(size):
    list_of_objects = list()
    for i in range(0,size):
        list_of_objects.append( list() ) #different object reference each time
    return list_of_objects

#========================== evaluate json image ===============================================
inference_array = init_list_of_objects(len(D))
for i in np.arange(1,len(D)-192,1):
  temp_img,temp_img4d = create_samples_from_idx(data['image'],int(i))
  temp_list = new_model.predict(temp_img4d)
  indexes = np.arange(int(i),int(i)+192,1)
  if (i % 200) == 0:   print(f"line {i} of {len(D)-192} is proccessed")
  for j in range(len(indexes)):
      # inference_array[indexes[j]].append(np.argmax(temp_list)+1)
      inference_array[indexes[j]].append(temp_list)

my_array = inference_array[192:(len(D)-192-1)]
inference_depth = D[192:(len(D)-192-1)]

final_inference=[]
for i in range(len(my_array)):
    temp = [0, 0, 0, 0, 0, 0]
    for j in range(192):
        for k in range(6):
            temp[k] = temp[k] + my_array[i][j][0][k]
    final_inference.append(np.argmax(temp)+1)

out_put=[[inference_depth[0],final_inference[0]]]
# temp = [inference_depth[0],final_inference[0]]
for i in range(1,len(final_inference)):
    out_put.append([inference_depth[i],final_inference[i]])


import pandas as pd
# print(Depth_array)
# df = pd.DataFrame(list(zip(Depth_arr,inference_arr)))
df = pd.DataFrame(list(np.array(out_put)))
with pd.ExcelWriter('inference_data.xlsx') as writer:
  df.to_excel(writer,sheet_name='Sheet1',index=False)



1/1 [==============================] - 0s 230ms/step
line 200 of 3508 is proccessed
1/1 [==============================] - 0s 240ms/step
line 400 of 3508 is proccessed
1/1 [==============================] - 0s 417ms/step
line 600 of 3508 is proccessed
1/1 [==============================] - 0s 236ms/step
line 800 of 3508 is proccessed
1/1 [==============================] - 0s 251ms/step
line 1000 of 3508 is proccessed
1/1 [==============================] - 0s 250ms/step
line 1200 of 3508 is proccessed
1/1 [==============================] - 0s 243ms/step
line 1400 of 3508 is proccessed
1/1 [==============================] - 0s 232ms/step
line 1600 of 3508 is proccessed
1/1 [==============================] - 0s 241ms/step
line 1800 of 3508 is proccessed
1/1 [==============================] - 0s 245ms/step
line 2000 of 3508 is proccessed
1/1 [==============================] - 0s 235ms/step
line 2200 of 3508 is proccessed
1/1 [==============================] - 0s 245ms/step
line 2400 of 350